# Question 6.1

## a)

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from itertools import product



# Partial CHATGPT Code
def recursively_remove_rows(X, y, original_X, original_y, indices_removed=[]):
    """
    Recursively tries to remove rows to improve or maintain the accuracy of a 1-NN classifier.

    :param X: The current dataset features from which rows are being removed.
    :param y: The current dataset labels from which corresponding rows are being removed.
    :param original_X: The original dataset features used for testing the classifier.
    :param original_y: The original dataset labels used for testing the classifier.
    :param indices_removed: Indices of rows that have been removed so far.
    :return: A tuple containing the pruned features and labels, and the indices of removed rows.
    """
    # Base case: if X is empty or all rows have been attempted for removal
    if len(X) == 0 or len(X) == len(indices_removed):
        return X, y, indices_removed

    for i in range(len(original_X)):
        if i in indices_removed:  # Skip already removed rows
            continue

        # Create a training dataset excluding the current row
        mask = np.array([index not in indices_removed and index != i for index in range(len(original_X))])
        X_train = original_X[mask]
        y_train = original_y[mask]

        # Train a 1-NN classifier
        knn = KNeighborsClassifier(n_neighbors=1)
        knn.fit(X_train, y_train)

        # Test on the entire original dataset
        predictions = knn.predict(original_X)
        accuracy = accuracy_score(original_y, predictions)

        # If accuracy is perfect, permanently remove the row and recurse
        if accuracy == 1:
            return recursively_remove_rows(X_train, y_train, original_X, original_y, indices_removed + [i])
        #else:
            #return len(X)

    # If no row can be removed to maintain perfect accuracy
    return X, y, indices_removed

/Users/seanzhou/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/seanzhou/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
/Users/seanzhou/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


We first test it on a binary dataset with $d = 2, 4, 5, 6$ and the associated $n = 4, 16, 32, 64$

In [4]:
d_vals = [2, 4, 5, 6]
num_of_class = 2

for d in d_vals:
    ns = []
    for i in range(50):
        # Permutate all combinations of features
        X = np.array(list(product([0, 1], repeat=d)))
        N, _ = X.shape
        y = np.random.randint(num_of_class, size=(N, 1)).ravel()
        X_final, y_final, rows_removed = recursively_remove_rows(X, y, X, y)
    #print(rows_removed)
        ns.append(N - len(rows_removed))    # Num of rows remaining
    n_avg = np.mean(ns)

    print(f"d={d}: n_full={2**d}, Avg. req. points for memorization n_avg={n_avg:.2f}, n_full/n_avg={(2**d)/n_avg}")

d=2: n_full=4, Avg. req. points for memorization n_avg=2.66, n_full/n_avg=1.5037593984962405
d=4: n_full=16, Avg. req. points for memorization n_avg=9.04, n_full/n_avg=1.7699115044247788
d=5: n_full=32, Avg. req. points for memorization n_avg=17.80, n_full/n_avg=1.797752808988764
d=6: n_full=64, Avg. req. points for memorization n_avg=34.56, n_full/n_avg=1.8518518518518516


We can see that `n_full/n_avg` tends to 2 as d increases

## b)

In [5]:
d_vals = [2, 4, 5, 6]
# Extend to multi-class
num_of_class = 3
for d in d_vals:
    ns = []
    for i in range(50):
        X = np.array(list(product([0, 1], repeat=d)))
        N, _ = X.shape
        y = np.random.randint(num_of_class, size=(N, 1)).ravel()
        X_final, y_final, rows_removed = recursively_remove_rows(X, y, X, y)
    #print(rows_removed)
        ns.append(N - len(rows_removed))
    n_avg = np.mean(ns)

    print(f"d={d}: n_full={2**d}, Avg. req. points for memorization n_avg={n_avg:.2f}, n_full/n_avg={(2**d)/n_avg}")

d=2: n_full=4, Avg. req. points for memorization n_avg=3.14, n_full/n_avg=1.2738853503184713
d=4: n_full=16, Avg. req. points for memorization n_avg=11.24, n_full/n_avg=1.4234875444839858
d=5: n_full=32, Avg. req. points for memorization n_avg=21.82, n_full/n_avg=1.466544454628781
d=6: n_full=64, Avg. req. points for memorization n_avg=42.84, n_full/n_avg=1.4939309056956114


As shown above, `n_full/n_avg` tends to 1.5 relatively quickly as d increases with 3 classes. This is expected, as $3/(3-1) = 1.5$ is the number of prediction bits in each parameter

# Question 6.2

## a)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


def find_split_points(x, labels):
    """Find if-else statements where the labels change"""
    split_points = []
    start = round(x[0],4)
    split_points.append((-np.inf, start, labels[0]))

    for i in range(1, len(x)):
        # Check if the label changes from the previous point
        if labels[i] != labels[i-1]:
            #print("change at ", i)
            end = round(x[i], 4)
            split_points.append((start, end, labels[i-1]))
            start = end
    
    split_points.append((start, np.Inf, labels[-1]))
    return split_points

def predict_if_else(x, split_points):
    """Predict the class for a given x based on split points."""
    x = float(x)
    for i in split_points:
        start = i[0]
        end = i[1]
        prediction = i[2]
        #print(start, end, prediction)
        if (x >= start) & (x < end):
            return prediction

def if_then_clauses(data, labels, weights = np.array([])):
    """Runs algorithm 8 and returns sorted_table and the thresholds (split_points)"""
    clauses = []
    n, d = data.shape
    df = pd.DataFrame(data).copy()
    if len(weights) == 0:
        df['sum'] = df.sum(axis=1)
    else:
        df['sum'] = np.dot(df, weights)
    df['label'] = labels
    sorted_table = df.sort_values(by='sum').reset_index(drop=True)
    #print(sorted_table[:100])

    #print(sorted_table)
    split_points = find_split_points(sorted_table['sum'].values, sorted_table['label'].values)

    return sorted_table, split_points

def eval_strategy(X, y, train_size, random_seed, tune=False, show_sorted=True, show_clauses=True):
    """Evaluate algorithm 8 results"""
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=random_seed)
    #print(X_train.shape, X_test.shape)
    if tune == False:
        weights = np.array([])
    else:
        model_no_intercept = LogisticRegression(fit_intercept=False).fit(X_train, y_train)
        # Model coefficients
        weights = model_no_intercept.coef_[0]
        
    sorted_train, clauses = if_then_clauses(X_train, y_train, weights=weights)
    sorted_test, _ = if_then_clauses(X_test, y_test, weights=weights)
    #print(sorted_test)
    predictions = [predict_if_else(x, clauses) for x in sorted_test['sum']]
    accuracy = accuracy_score(y_test, predictions)

    if show_clauses:
        print("if-else clauses: ")
        print(clauses[1:-1])
    print("Number of clauses: ", len(clauses[1:-1]))
    if show_sorted==True:
        print("Sorted_dataset: ")
        print(sorted_test.head(20))
    print("Prediction accuracy on test set: ", accuracy)
    return None

Here we demonstrate how our algorithm works using a simple dataset

In [7]:
# Example data
x_values = [1, 2, 3, 5, 7, 9]
labels = [0, 0, 1, 0, 1, 1]

# Find the split points
split_points = find_split_points(x_values, labels)
test_x_values = [0, 2, 4, 6, 8, 10]
predictions = [predict_if_else(x, split_points) for x in test_x_values]

print(split_points)
print("y_values: ")
print(labels)
print("Predicted: ")
print(predictions)

[(-inf, 1, 0), (1, 3, 0), (3, 5, 1), (5, 7, 0), (7, inf, 1)]
y_values: 
[0, 0, 1, 0, 1, 1]
Predicted: 
[0, 0, 1, 0, 1, 1]


In [8]:
# Load the Iris dataset
iris = sns.load_dataset("iris")

binary_iris = iris[(iris['species'] == 'setosa') | (iris['species'] == 'versicolor')]

# Convert 'species' to a binary variable: 'setosa' as 0, 'versicolor' as 1
binary_iris['species'] = binary_iris['species'].map({'setosa': 0, 'versicolor': 1})

/var/folders/5w/gsy5p7694w76q69v5jwrqgl40000gn/T/ipykernel_56616/60707310.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binary_iris['species'] = binary_iris['species'].map({'setosa': 0, 'versicolor': 1})


We first apply algorithm 8 on the `iris` dataset with binary outcomes to minimize the number of if-else statements without tuning the weights

In [9]:
# Without applying weights
X = binary_iris.iloc[:, :-1]
y = binary_iris.iloc[:, -1]

# Generating a dataset for the AND function
# Define the inputs and the corresponding outputs for AND function
# inputs = [(0, 0), (0, 1), (1, 1), (1, 0)]
# outputs = [0, 0, 1, 0]

# df_and = pd.DataFrame(inputs, columns=['x1', 'x2'])
# df_and['y'] = outputs

# X = df_and.iloc[:, :-1]
# y = df_and.iloc[:, -1]

eval_strategy(X, y, 0.8, random_seed=3, tune=False)

if-else clauses: 
[(8.4, 11.5, 0), (11.5, 11.5, 1), (11.5, 11.6, 0)]
Number of clauses:  3
Sorted_dataset: 
    sepal_length  sepal_width  petal_length  petal_width   sum  label
0            4.4          3.2           1.3          0.2   9.1      0
1            4.8          3.0           1.4          0.3   9.5      0
2            4.9          3.1           1.5          0.1   9.6      0
3            4.9          3.1           1.5          0.2   9.7      0
4            4.6          3.4           1.4          0.3   9.7      0
5            5.0          3.0           1.6          0.2   9.8      0
6            5.1          3.3           1.7          0.5  10.6      0
7            5.0          2.3           3.3          1.0  11.6      1
8            5.1          2.5           3.0          1.1  11.7      1
9            5.7          4.4           1.5          0.4  12.0      0
10           5.2          2.7           3.9          1.4  13.2      1
11           5.6          2.9           3.6         

Now we try to tune the weights using logistic regression without the intercept:

In [10]:
eval_strategy(X, y, 0.8, random_seed=3, tune=True)

if-else clauses: 
[(-5.365, 2.1947, 0)]
Number of clauses:  1
Sorted_dataset: 
    sepal_length  sepal_width  petal_length  petal_width       sum  label
0            5.7          4.4           1.5          0.4 -5.075433      0
1            4.6          3.4           1.4          0.3 -3.506192      0
2            4.4          3.2           1.3          0.2 -3.440568      0
3            4.9          3.1           1.5          0.1 -3.180065      0
4            4.9          3.1           1.5          0.2 -3.090462      0
5            4.8          3.0           1.4          0.3 -3.028316      0
6            5.0          3.0           1.6          0.2 -2.781923      0
7            5.1          3.3           1.7          0.5 -2.766738      0
8            5.1          2.5           3.0          1.1  1.627045      1
9            5.6          2.9           3.6          1.3  2.294220      1
10           5.0          2.3           3.3          1.0  2.491092      1
11           5.8          2.7    

We see that we reduced the number of thresholds from 3 to 1 while increasing our prediction accuracy on the test set. 

In [11]:
### Strategy 1: 1 if-else statement
### Predict majority class for all rows
# def predict_majority_class(X, y):
#     majority_class = y.mode()[0]
#     predictions = np.full(len(X), majority_class, dtype=int)

#     accuracy = accuracy_score(predictions, y)

#     return predictions, accuracy

### Strategy 1: sum of xs > b
# def predict_sum(X, y):
#     max_b = X.shape[1]
#     best_acc = 0
#     best_b = 0
#     best_pred = []
#     for i in range(max_b):
#         feature_sum = X.sum(axis=1)
#         # Predict 1 if the sum is greater than 'b', else 0
#         predictions = (feature_sum > i).astype(int)

#         accuracy_temp = accuracy_score(predictions, y)
#         if accuracy_temp > best_acc:
#             best_acc = accuracy_temp
#             best_b = i
#             best_pred = predictions
    
#     return predictions, best_acc, best_b

# def do_algorithm_8(X, y, weights = np.array([])):
#     threshold = 0
#     n, d = X.shape
    
#     table = pd.DataFrame()
#     if len(weights) == 0:
#         table['sum_x'] = X.sum(axis=1)
#     else:
#         table['sum_x'] = np.dot(X, weights)
#     table['y'] = y
#     table_sorted = table.sort_values(by="sum_x")
#     #print(table_sorted)
#     print(table_sorted)

#     #class_ = np.zeros(n)
#     # for i in range(n):
#     #     y_row = table_sorted.iloc[i, 1]
#     #     # if y is not 0, then 
#     #     if table_sorted.iloc[i, 1] != 0:
#     #         class_[i] = y_row
#     #         threshold += 1
#     ys_sorted = table_sorted.iloc[:, -1].values
#     threshold = 0
    
#     for i in range(len(ys_sorted) - 1):
#         if ys_sorted[i] != ys_sorted[i + 1]:
#             # Increment the counter if a change is found
#             threshold += 1
            
#     # num of threshold comparisons = num of if-else statements 
#     #print("Threshold", threshold)
    
#     minthres = np.log2(threshold + 1)
#     mec = int((minthres * (d+1)) + (minthres + 1))

#     return threshold, minthres, mec

## b)

Now we try our algorithms on the `xor` dataset and the famous `titanic` dataset (only binary explanatory variables are retained)

In [12]:
# Generating a dataset for the Xor function
# Define the inputs and the corresponding outputs for XOR function
inputs = [(0, 0), (0, 1), (1, 0), (1, 1)]
outputs = [0, 1, 1, 0]

# Create a DataFrame
df_xor = pd.DataFrame(inputs, columns=['x1', 'x2'])
df_xor['y'] = outputs

df_xor

,x1,x2,y
0,0,0,0
1,0,1,1
2,1,0,1
3,1,1,0


In [13]:
X = df_xor.iloc[:, :-1]
y = df_xor.iloc[:, -1]

eval_strategy(X, y, train_size=0.8, random_seed=12, tune=False, show_sorted=False)

if-else clauses: 
[(1, 2, 1)]
Number of clauses:  1
Prediction accuracy on test set:  0.0


In [14]:
X = df_xor.iloc[:, :-1]
y = df_xor.iloc[:, -1]

eval_strategy(X, y, train_size=0.8, random_seed=12, tune=True, show_sorted=False)

if-else clauses: 
[(0.0, 0.0, 1)]
Number of clauses:  1
Prediction accuracy on test set:  1.0


In [15]:
# Load the Titanic dataset
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [16]:
df = titanic.loc[:, ["survived", "sibsp", "adult_male", "who", "alone"]]
# Convert boolean columns to integers
df = pd.get_dummies(df, columns=["adult_male", "who", "alone"], drop_first=True)
df = df.drop("who_woman", axis=1)

# Convert boolean columns to integers (0s and 1s)
df['adult_male'] = df['adult_male_True'].astype(int)
df['who'] = df['who_man'].astype(int)
df['alone'] = df['alone_True'].astype(int)

# Drop the original boolean columns if they are no longer needed
df.drop(columns=['adult_male_True', 'who_man', 'alone_True'], inplace=True)

# Now df has the columns with 0s and 1s instead of True/False.
df.head()

,survived,sibsp,adult_male,who,alone
0,0,1,1,1,0
1,1,1,0,0,0
2,1,0,0,0,1
3,1,1,0,0,0
4,0,0,1,1,1


In [17]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

eval_strategy(X, y, train_size=0.8, random_seed=12, tune=False, show_clauses=False, show_sorted=True)

Number of clauses:  225
Sorted_dataset: 
    sibsp  adult_male  who  alone  sum  label
0       0           0    0      0    0      1
1       0           0    0      0    0      0
2       0           0    0      0    0      1
3       0           0    0      0    0      0
4       0           0    0      0    0      1
5       0           0    0      0    0      1
6       0           0    0      0    0      1
7       0           0    0      0    0      1
8       0           0    0      0    0      1
9       0           0    0      0    0      0
10      0           0    0      0    0      1
11      1           0    0      0    1      1
12      0           0    0      1    1      1
13      0           0    0      1    1      1
14      1           0    0      0    1      1
15      0           0    0      1    1      0
16      0           0    0      1    1      1
17      1           0    0      0    1      1
18      1           0    0      0    1      1
19      1           0    0      0    1 

In [18]:
eval_strategy(X, y, train_size=0.8, random_seed=12, tune=True, show_clauses=False, show_sorted=False)

Number of clauses:  198
Prediction accuracy on test set:  0.5586592178770949


Using the `XOR` dataset, we observe that tuning the weights did not reduce the number of clauses but improved the overall accuracy significantly. 

Using the binary features in the titanic dataset, we see that the prediciton accuracy still improved when we decided to tune our weights, but the clauses are messed up due to violations of the functional assumption of supervised machine learning: there are many people with the same features ($x_i = x_j$) but different survival statuses ($f(x_i) \neq f(x_j)$). 

## c)

In [19]:
# CHATGPT Code
def create_random_binary_dataset(num_samples=100, num_features=3, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
    
    # Generate random binary values for the features
    features = np.random.randint(2, size=(num_samples, num_features))
    
    # Generate a random binary target variable
    target = np.random.randint(2, size=(num_samples, 1))
    
    # Create a DataFrame
    data = pd.DataFrame(features, columns=[f'x{i+1}' for i in range(num_features)])
    data['y'] = target
    
    return data

In [20]:
# When n >> d (n=100, d=3)
df_long = create_random_binary_dataset(num_samples = 100, num_features = 50,random_state=1)

# When n = d (n=d=100)
df_equal = create_random_binary_dataset(num_samples = 100, num_features = 100, random_state=1)

# When n << d (n=3, d=100)
df_wide = create_random_binary_dataset(num_samples = 50, num_features = 100, random_state=1)

In [21]:
for df in [df_long, df_equal, df_wide]:
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    print("Results with no weight tuning:")
    print("--------------------------------------------------")
    eval_strategy(X, y, train_size=0.8, random_seed=1, show_sorted=False, show_clauses=False)
    print("Results with weight tuning:")
    print("--------------------------------------------------")
    eval_strategy(X, y, train_size=0.8, random_seed=1, tune=True, show_sorted=False, show_clauses=False)
    # threshold, minthres, mec = do_algorithm_8(df)
    # print("Num of if-else statements: ", threshold)
    # print("Minimum amount of thresholds: ", minthres)


Results with no weight tuning:
--------------------------------------------------
Number of clauses:  33
Prediction accuracy on test set:  0.55
Results with weight tuning:
--------------------------------------------------
Number of clauses:  13
Prediction accuracy on test set:  0.5
Results with no weight tuning:
--------------------------------------------------
Number of clauses:  35
Prediction accuracy on test set:  0.6
Results with weight tuning:
--------------------------------------------------
Number of clauses:  1
Prediction accuracy on test set:  0.7
Results with no weight tuning:
--------------------------------------------------
Number of clauses:  22
Prediction accuracy on test set:  0.6
Results with weight tuning:
--------------------------------------------------
Number of clauses:  1
Prediction accuracy on test set:  0.5


Here not only do we change the $n$ and $d$ to make a long, wide and square data matrix, we also try to see if tuning the weights improve our algorithm performance on a completely random dataset. We observe no significant differences in terms of the prediction accuracy with or without weights tuning. This is expected as our dataset is completely random with no underlying patterns

## Question 6.3

In [22]:
import random
import zlib

In [23]:
def generate_random_string(length=1024):
    # Using a set of characters to generate the random string
    chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789'
    return ''.join(random.choice(chars) for _ in range(length))

def compress_string(input_string):
    # Compressing the input string using gzip
    compressed_data = zlib.compress(input_string.encode('utf-8'))
    return compressed_data

In [24]:
# An example of char string
print(generate_random_string()[:20])

botFX8G13t44fDMyOyKZ


In [25]:
# string lengths to test
string_lengths = [200, 1000, 10000, 100000]

results = []

# Loop through each string length
for length in string_lengths:
    random_string = generate_random_string(length)
    compressed_string = compress_string(random_string)
    # Calculate the compression ratio
    compression_ratio = len(compressed_string) / len(random_string.encode('utf-8'))
    results.append({"String Length": length, "Compression Ratio": compression_ratio})

# Convert the results list to a DataFrame
compression_summary = pd.DataFrame(results)

In [26]:
compression_summary

,String Length,Compression Ratio
0,200,0.9250
1,1000,0.7790
2,10000,0.7527
3,100000,0.7519


Typically, we expect the compression ratio of applying a lossless compression on random string to be close to 1 or even exceeds 1. This is because long random strings have very high entropy, and our lossless compression algorithm struggles to find redundant or repeating patterns in our string. However, here we got a much lower number for long random strings, potentially because 1. the cost of overheads is relatively smaller and 2. there are more opportunities for pattern detection for larger strings.


# Question 8.1

## a)

12 + min(12, 3) + min(12, 3) = 18 bits

## b)

3 + 4 + 4 = 11 bits

## c)

can memorize 18 rows and b) can memorize 11 rows

## d)

$log_2 4 = 2$. 

$18 // 2 = 9$ and $11 // 2 = 5$

# Question 8.2 

![Alt text](Untitled.png "The Two NN Architechtures")

# Question 8.4

## a)

## Estimation of Total Sensory Experience

Assume the human eye is a camera with $576$ megapixels. Each pixel has $24$ bits of color depth ($8$ bits for each of the red, green, and blue channels). Therefore the information per second can be calculated as $576 \times 24 = 13,824$ megabits per second. This is $1.4 \times 10^{10}$ bits

Then, the total seconds in 24 years assuming 16 hours of wakefulness per day is around $10^9$ seconds. Thus the total amount of information of visual experience in 24 years is $1.4 \times 10^{19}$ bits. Similar analysis can be done for auditory information. Let total visual experience be $X$ and total sensory experience be $Y$ and assuming complete independence, the final capacity is $H(X) + H(Y) \approx 3 \times 10^{19}$ bits

## How much have we memorized?

Let's assume on average humans remember only 20% of auditory and visual information. Then that would be $6 * 10^{18}$ bits

## Capacity of human brain

We can ignore the bias term due to the great size of the number of neurons: MEC of the brain is $10^{11} \times 1000 = 10^{14}$ bits. Still this shows that our brain would be  full if we memorize 20% of all input information, as $10^{14} << 10^{18}$ bits that we estimated

## Shakespeare

If we assume 6 bytes per word including spaces and punctuations, Shakespeare's collected works include a bit under 1 million words of texts in total. Then assuming there is no mutual information contained between words, the total number of information is additive: $6 \times 1,000,000 = 6 \text{mb}$ of info ([source](https://nlp.stanford.edu/IR-book/html/htmledition/an-example-information-retrieval-problem-1.html))

## b)

We can use algorithm 8 and train it on a random dataset with c equi-distributed classes. Then divide the number of thresholds generated by the number of instances memorized. We should expect to see $\frac{c}{c-1}$

In [27]:
# def do_algorithm_8_multiclass(df, k):
#     """
#     Assumes classes are indexed as 0, 1, 2....k-1 where k is the number of classes
#     """
#     threshold = 0
#     n = df.shape[0]
#     d = df.shape[1] - 1   # number of columns
    
#     table = pd.DataFrame()
#     table['sum_x'] = df.iloc[:, :-1].sum(axis=1)
#     table['y'] = df.y
#     table_sorted = table.sort_values(by="sum_x")
#     #print(table_sorted)

#     class_ = np.zeros(n)
#     for i in range(n):
#         y_row = table_sorted.iloc[i, 1]
#         # if y is not 0, then 
#         if table_sorted.iloc[i, 1] != 0:
#             class_[i] = y_row
#             threshold += 1
    
#     # num of threshold comparisons = num of if-else statements 
#     #print("Threshold", threshold)
   
#     minthres =  math.log(threshold + 1, k)

#     mec = int((minthres * (d+1)) + (minthres + 1))

#     return threshold, minthres, mec

## c) 

Similar argument as above, except we should see $1$ as the resulting number, as $\frac{c}{c-1}$ equals $1$ as c tends to $\infty$ when we have regression

In [28]:
# def do_algorithm_8_reg (df): 
#     """
#     Runs algorithm 8 for regression tasks
#     """
#     threshold = 0
#     n = df.shape[0]
#     d = df.shape[1] - 1   # number of columns
    
#     table = pd.DataFrame()
#     table['sum_x'] = df.iloc[:, :-1].sum(axis=1)
#     table['y'] = df.y
#     table_sorted = table.sort_values(by="sum_x")
#     print(table_sorted)

# def generate_reg_data(n, d):
#     """
#     Generates a completely random regression dataset.
#     """
#     # Generate random features
#     X = np.random.randn(n, d)
#     coefficients = np.random.randn(d, 1)
    
#     # Generate noise
#     noise = np.random.randn(n, 1)
    
#     # Generate target variable y = X * coefficients + noise
#     y = X @ coefficients + noise
#     y = y.flatten()
#     df = pd.DataFrame(X, columns=[f'x_{i}' for i in range(1, d+1)])
    
#     # Add the target variable column to the DataFrame
#     df['y'] = y
    
#     return df

# # Example usage
# n = 100  # Number of samples
# d = 5    # Number of features
# df = generate_reg_data(n, d)
# df


## Question 9.6

SEE OTHER NOTEBOOK